In [ ]:
import astropy.units as u
import os
import pandas as pd
import galpy.util.coords 
import galpy
import sys
sys.modules['galpy.util.bovy_coords'] = galpy.util.coords
import math
from astropy.coordinates import SkyCoord

In [5]:
os.chdir('/Users/mlclark/Library/CloudStorage/GoogleDrive-mclark.astro@gmail.com/My Drive/UMa/final_uma/wdwarf_files')

In [ ]:
uma_wd=pd.read_csv('25_6_30_wds_w_ages.csv', dtype={'GaiaID':str}) #whatever sources you want offset errs for
r=pd.DataFrame()
r['pmdec']=uma_wd.pmdec
r['pmdra']=uma_wd.pmra
r['dec']=uma_wd.dec
r['ra']=uma_wd.ra
r['parallax']=uma_wd.parallax

In [ ]:
#replace with target star info from gaia. this is HD 115043
Pcoord = SkyCoord( ra=198.405103929*u.deg , dec= 56.7081841459*u.deg , \
                    distance=(1000.0/39.7437)*u.parsec , frame='icrs' , \
                    radial_velocity=-8.56*u.km/u.s , \
                    pm_ra_cosdec=111.5498*u.mas/u.year , pm_dec=-18.5278*u.mas/u.year )   
gaiacoord = SkyCoord( ra=r['ra'].values*u.deg , dec=r['dec'].values*u.deg.to(u.rad)*u.rad , distance=(1000.0/r['parallax'].values)*u.parsec , \
                        frame='icrs' , \
                        pm_ra_cosdec=r['pmra'].values*u.mas/u.year , pm_dec=r['pmdec'].values*u.mas/u.year )
sep = gaiacoord.separation(Pcoord)
sep3d = gaiacoord.separation_3d(Pcoord)
Pllbb     = bc.radec_to_lb(Pcoord.ra.value , Pcoord.dec.value , degree=True)
Ppmllpmbb = bc.pmrapmdec_to_pmllpmbb( Pcoord.pm_ra_cosdec.value , Pcoord.pm_dec.value , \
                                         Pcoord.ra.value , Pcoord.dec.value , degree=True )
Pvxvyvz   = bc.vrpmllpmbb_to_vxvyvz(Pcoord.radial_velocity.value , Ppmllpmbb[0] , Ppmllpmbb[1] , \
                                   Pllbb[0] , Pllbb[1] , Pcoord.distance.value/1000.0 , XYZ=False , degree=True)
Cll = (math.atan2(Pvxvyvz[1],Pvxvyvz[0]) * 180.0/np.pi) % 360
Cbb = math.atan2(Pvxvyvz[2],np.sqrt(Pvxvyvz[0]**2+Pvxvyvz[1]**2)) * 180.0/np.pi

Cradec = bc.lb_to_radec(Cll,Cbb,degree=True,epoch=2000.0)
Ccoord = SkyCoord( ra=Cradec[0]*u.deg , dec=Cradec[1]*u.deg , distance=999999.9 , frame='icrs' )
Cangle = gaiacoord.separation(Ccoord)
zz = np.where( (Cangle.degree > 90.0) )
if (np.array(zz).size > 0): Cangle[zz] = (180.0-Cangle[zz].degree)*u.deg